In [1]:
from fastapi import FastAPI

In [3]:
from pydantic import BaseModel

# import nltk
# import sklearn

app = FastAPI()


class Question(BaseModel):
    user_input:str

In [9]:
@app.post("/")
async def answer_endpoint(input_q: Question):
    question = input_q.dict()
 

    return {
        "user_input": question["user_input"],
    }

@app.get("/")
async def root():
    return {
        "message": "this is get"
    }

In [1]:
from fastapi import FastAPI
from pydantic import BaseModel, Field
from typing import Optional
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores import Chroma

# import sklearn

app = FastAPI()


class Question(BaseModel):
    user_input:str
    k: Optional[int] = Field(default=3)


@app.post("/")
async def retrieve_docs(input_q: Question):
    question = input_q.dict()
    # numerical = str_to_numerical.transform([question["user_input"]]).toarray()
    # distance, idx = info_retrieve.query(numerical, k=1)

    # best_answer = list(enumerate(idx[0]))[0]
    instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="/root/.cache/torch/sentence_transformers/GanymedeNil_text2vec-large-chinese",
                                                      # local_files_only=True,
                                                      model_kwargs={"device": "cuda"})
    persist_directory = 'db_cn'

    vectordb = Chroma(persist_directory=persist_directory,
                      embedding_function=instructor_embeddings)

    retriever = vectordb.as_retriever(search_kwargs={"k": question['k']})

    ans = retriever.get_relevant_documents(question["user_input"])
    docs = list(map(lambda a: {
            'page_content': a.page_content,
            'source': a.metadata["source"],
            'page': a.metadata["page"]}, ans))
    return {
        "user_input": question["user_input"],
        "relavant_docs": docs
    }

@app.get("/")
async def root():
    return {
        "message": "this is get"
    }

In [2]:
import nest_asyncio

# Allow for asyncio to work within the Jupyter notebook cell
nest_asyncio.apply()

In [3]:
import uvicorn
uvicorn.run(app)

INFO:     Started server process [5903]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
/usr/local/lib/python3.10/site-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange
No sentence-transformers model found with name /root/.cache/torch/sentence_transformers/GanymedeNil_text2vec-large-chinese. Creating a new one with MEAN pooling.
2023-09-05 14:33:26.249469: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-05 14:33:27.040586: W tensorflow/compiler/tf2tensor

INFO:     127.0.0.1:49452 - "POST / HTTP/1.1" 200 OK


No sentence-transformers model found with name /root/.cache/torch/sentence_transformers/GanymedeNil_text2vec-large-chinese. Creating a new one with MEAN pooling.


INFO:     127.0.0.1:49454 - "POST / HTTP/1.1" 200 OK


No sentence-transformers model found with name /root/.cache/torch/sentence_transformers/GanymedeNil_text2vec-large-chinese. Creating a new one with MEAN pooling.


INFO:     127.0.0.1:49456 - "POST / HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [5903]


In [1]:
%run use_endpoint.py

b'{"user_input":"What is the work station?"}'
b'{"message":"this is get"}'
